In [1]:
%%capture
!pip install numpy==1.21.4
!pip install pandas==1.3.4
!pip install loguru==0.5.3

In [4]:
import sys
import os
from tqdm import tqdm
regions = [i for i in os.listdir() if (len(i) == 2) and (not i in ['88', '81', '82', '80', '84', '85'])]
data_dir = './'
final_dir = './result'

In [ ]:
for r in regions:
    if (not f'{r}_hadobj.csv' in os.listdir(final_dir)) or (not f'{r}_parsed_chains.csv' in os.listdir(final_dir)):
        %run gar_try.py {data_dir} {r} {final_dir}

./ ./43 ./result
chready: True


100%|██████████| 328256/328256 [00:01<00:00, 169988.51it/s]


./ ./30 ./result
chready: True


100%|██████████| 265600/265600 [00:01<00:00, 176853.54it/s]


./ ./91 ./result
chready: True


...

./ ./13 ./result
chready: True


100%|██████████| 241046/241046 [00:01<00:00, 184766.41it/s]


./ ./67 ./result
chready: True


100%|██████████| 264580/264580 [00:01<00:00, 173874.93it/s]


./ ./06 ./result


In [6]:
#%run gar_try.py {data_dir} 01 {final_dir}

In [8]:
done = 0
for r in tqdm(regions):
    if (f'{r}_hadobj.csv' in os.listdir(final_dir)) and (f'{r}_parsed_chains.csv' in os.listdir(final_dir)):
        done +=1
done

100%|██████████| 86/86 [00:00<00:00, 45997.21it/s]


1